In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import glob
from tqdm.auto import tqdm


In [ ]:
list_path_csv = glob.glob('./data/crime_raw/*.csv')
list_df_crime = [pd.read_csv(path_csv) for path_csv in list_path_csv]


In [5]:
# Examine sample structure from already loaded data
df_sample = list_df_crime[0]
print(f"Sample file: {list_path_csv[0]}")
print(df_sample.head())


Sample file: ./data/crime/Aggravated Assault Reported by Population_10-15-2025 (26).csv
                series  10-2015  11-2015  12-2015  01-2016  02-2016  03-2016  \
0             Missouri     1798     1813     1637     1691     1428     1786   
1  Missouri Clearances      939      951      857      902      811      954   

   04-2016  05-2016  06-2016  ...  01-2025  02-2025  03-2025  04-2025  \
0     1866     1945     2074  ...     1293     1158     1674     1801   
1      879     1006      963  ...      612      604      771      758   

   05-2025  06-2025  07-2025  08-2025  09-2025  10-2025  
0     1925     1815     1719     1376       93      NaN  
1      793      746      758      572       53      NaN  

[2 rows x 122 columns]


In [6]:
list_df_agg = []

with tqdm(list_path_csv, leave=True, position=0) as pbar:
    for path_csv in pbar:
        # Extract crime type from filename
        str_filename = Path(path_csv).stem
        str_crime = str_filename.split('_')[0]
        
        # Read CSV
        df_current = pd.read_csv(path_csv)
        
        # Separate state rows and clearance rows
        df_states = df_current[~df_current['series'].str.contains('Clearances', na=False)].copy()
        df_clearances = df_current[df_current['series'].str.contains('Clearances', na=False)].copy()
        
        # Clean clearances state names
        df_clearances['series'] = df_clearances['series'].str.replace(' Clearances', '')
        
        # Melt states data
        df_states_long = df_states.melt(id_vars=['series'], var_name='date', value_name='count')
        df_states_long.rename(columns={'series': 'state'}, inplace=True)
        
        # Melt clearances data
        df_clearances_long = df_clearances.melt(id_vars=['series'], var_name='date', value_name='Clearances')
        df_clearances_long.rename(columns={'series': 'state'}, inplace=True)
        
        # Merge states and clearances
        df_merged = pd.merge(df_states_long, df_clearances_long, on=['state', 'date'], how='left')
        
        # Add crime column
        df_merged['crime'] = str_crime
        
        # Reorder columns: series (date), state, crime, Clearances
        df_merged = df_merged[['date', 'state', 'crime', 'count', 'Clearances']]
        df_merged.rename(columns={'date': 'series'}, inplace=True)
        
        list_df_agg.append(df_merged)

# Concatenate all dataframes
df_crime_agg = pd.concat(list_df_agg, ignore_index=True)


  0%|          | 0/57 [00:00<?, ?it/s]

In [9]:
df_crime_agg.head(10)
df_crime_agg.to_csv("data/crime/crime_aggregated.csv", index=False)
